In [2]:
import graphlab as gl
gl.canvas.set_target('ipynb')

In [13]:
song_data = gl.SFrame('song_data.gl/')
song_data.column_names()

['user_id', 'song_id', 'listen_count', 'title', 'artist', 'song']

## Counting unique users

In [22]:
def unique_listeners_for_artist(artist):
    users_for_artist = song_data[song_data['artist'] == artist]['user_id'].unique()
    return len(users_for_artist)

def print_unique_listeners(artist):
    print("Unique listeners for {}: {}".format(artist, unique_listeners_for_artist(artist)))

print_unique_listeners('Kanye West')
print_unique_listeners('Foo Fighters')
print_unique_listeners('Taylor Swift')
print_unique_listeners('Lady GaGa')
    

Unique listeners for Kanye West: 2522
Unique listeners for Foo Fighters: 2055
Unique listeners for Taylor Swift: 3246
Unique listeners for Lady GaGa: 2928


## Most and least popular artist

In [44]:
popularities = song_data \
    .groupby(key_columns='artist', operations={'total_count': gl.aggregate.SUM('listen_count')}) \
    .sort("total_count", ascending=False)

print ("Most popular: " + str(popularities[0]))
print ("Least popular: " + str(popularities[-1]))

Most popular: {'total_count': 43218, 'artist': 'Kings Of Leon'}
Least popular: {'total_count': 14, 'artist': 'William Tabbert'}


## Most recommended songs

#### Fist, train the model

In [48]:
train_data,test_data = song_data.random_split(.8,seed=0)
model = gl.item_similarity_recommender.create(train_data,
                                                 user_id='user_id',
                                   b              item_id='song')

Recsys training: model = item_similarity

Warning: Ignoring columns song_id, listen_count, title, artist;

To use one of these as a target column, set target = <column_name>

and use a method that allows the use of a target.

Preparing data set.

...

| 7000            | 0.806928        |

| 8000            | 0.846249        |

| 9000            | 0.89324         |

+-----------------+-----------------+

Finished training in 1.27906s

#### Then, check the resultst on subset of users

In [51]:
test_users = test_data['user_id'].unique()[0:10000]
recommendations = model.recommend(test_users,k=1)

recommendations.groupby(key_columns='song', operations={'count': gl.aggregate.COUNT()}).sort('count', ascending=False)

recommendations finished on 1000/10000 queries. users per second: 1719.48

recommendations finished on 2000/10000 queries. users per second: 1653.58

recommendations finished on 3000/10000 queries. users per second: 1646.8

recommendations finished on 4000/10000 queries. users per second: 1658.5

recommendations finished on 5000/10000 queries. users per second: 1668.49

...

recommendations finished on 7000/10000 queries. users per second: 1694.38

recommendations finished on 8000/10000 queries. users per second: 1697.16

recommendations finished on 9000/10000 queries. users per second: 1675.48

recommendations finished on 10000/10000 queries. users per second: 1679.79

song,count
Undo - Björk,438
Secrets - OneRepublic,370
Revelry - Kings Of Leon,238
You're The One - DwightYoakam ...,163
Fireflies - CharttraxxKaraoke ...,118
Hey_ Soul Sister - Train,108
Horn Concerto No. 4 in Eflat K495: II. Romance ...,89
Sehr kosmisch - Harmonia,85
OMG - Usher featuringwill.i.am ...,63
Clocks - Coldplay,48
